In [1]:
import os
cwd = os.getcwd()
data_dir = cwd + '/dataset'
os.listdir(data_dir)

['prev_check2_clean.csv',
 'application_test.csv',
 '.DS_Store',
 'HomeCredit_columns_description.csv',
 'POS_CASH_balance.csv',
 'credit_card_balance.csv',
 'installments_payments.csv',
 'application_train.csv',
 'bureau.csv',
 'bu_clean.csv',
 'prev_check1_clean.csv',
 'previous_application.csv',
 'HomeCredit_columns_description.xls',
 'bureau_balance.csv',
 'sample_submission.csv']

In [2]:
import numpy as np
import pandas as pd
import plotly.offline as py
import plotly.graph_objs as go
import plotly.tools as tls
pd.set_option('max_columns',500)
pd.set_option('max_rows',500)

In [3]:
prev_appli_data = pd.read_csv(data_dir + '/previous_application.csv')
instal_pay_data = pd.read_csv(data_dir + '/installments_payments.csv')

In [4]:
#print(set(prev_appli_data["NAME_CONTRACT_TYPE"]))
prev_appli_data.head(3)

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,1,0.0,0.182832,0.867336,XAP,Approved,-73,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-164,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,-1,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-301,Cash through the bank,XAP,"Spouse, partner",Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0


In [5]:
#prev_appli_data.loc[prev_appli_data["SK_ID_CURR"]==108129]

In [6]:
print(set(prev_appli_data["NAME_CONTRACT_STATUS"].values))
#prev_appli_data.loc[prev_appli_data["NAME_CONTRACT_STATUS"]=='Unused offer'].head(5)

{'Refused', 'Approved', 'Canceled', 'Unused offer'}


In [7]:
print(set(prev_appli_data.loc[prev_appli_data["NAME_CONTRACT_STATUS"]=='Refused',"CODE_REJECT_REASON"].values))
print(set(prev_appli_data["NAME_CLIENT_TYPE"].values))
print(set(prev_appli_data["NAME_PORTFOLIO"].values))
print(set(prev_appli_data["NAME_PRODUCT_TYPE"].values))
print(set(prev_appli_data["CHANNEL_TYPE"].values))
print(set(prev_appli_data["NAME_YIELD_GROUP"].values))
print(set(prev_appli_data["PRODUCT_COMBINATION"].values))

{'XAP', 'SCO', 'LIMIT', 'XNA', 'SCOFR', 'SYSTEM', 'HC', 'VERIF'}
{'Refreshed', 'New', 'Repeater', 'XNA'}
{'Cash', 'XNA', 'Cars', 'POS', 'Cards'}
{'walk-in', 'XNA', 'x-sell'}
{'Channel of corporate sales', 'Country-wide', 'Regional / Local', 'Credit and cash offices', 'Car dealer', 'AP+ (Cash loan)', 'Stone', 'Contact center'}
{'XNA', 'high', 'low_action', 'middle', 'low_normal'}
{'Cash X-Sell: high', 'Cash X-Sell: middle', 'Cash Street: high', 'POS household with interest', 'Card X-Sell', 'Cash Street: middle', 'Cash X-Sell: low', nan, 'POS other with interest', 'POS mobile without interest', 'Cash Street: low', 'POS others without interest', 'POS household without interest', 'POS industry without interest', 'POS mobile with interest', 'Cash', 'POS industry with interest', 'Card Street'}


In [8]:
set(prev_appli_data.loc[prev_appli_data["NAME_CONTRACT_STATUS"]=='Unused offer',\
                        "NFLAG_INSURED_ON_APPROVAL"].apply(str).values)

{'nan'}

In [9]:
instal_pay_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13605401 entries, 0 to 13605400
Data columns (total 8 columns):
SK_ID_PREV                int64
SK_ID_CURR                int64
NUM_INSTALMENT_VERSION    float64
NUM_INSTALMENT_NUMBER     int64
DAYS_INSTALMENT           float64
DAYS_ENTRY_PAYMENT        float64
AMT_INSTALMENT            float64
AMT_PAYMENT               float64
dtypes: float64(5), int64(3)
memory usage: 830.4 MB


In [10]:
#instal_pay_data.loc[instal_pay_data["SK_ID_PREV"]==2802425].head(10)

In [11]:
prev_appli_data.loc[prev_appli_data["SK_ID_PREV"]==1740877]

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
161591,1740877,371185,Revolving loans,5625.0,0.0,112500.0,NaN,NaN,TUESDAY,9,Y,1,NaN,NaN,NaN,XAP,Approved,-1108,XNA,XAP,NaN,Repeater,XNA,Cards,x-sell,Credit and cash offices,-1,XNA,0.0,XNA,Card X-Sell,-1106.0,-1082.0,365243.0,365243.0,365243.0,0.0


## 清理数据

** 整理每个id_curr下的approved 与refused 的贷款数量**

In [12]:
df = prev_appli_data[["SK_ID_CURR",\
                      "NAME_CONTRACT_STATUS",\
                      "NAME_CONTRACT_TYPE"]\
                    ].groupby(["SK_ID_CURR",\
                               "NAME_CONTRACT_STATUS"\
                              ]).count()
df = df.unstack(1)

In [13]:
df.head(3)

NAME_CONTRACT_TYPE                              
NAME_CONTRACT_STATUS           Approved Canceled Refused Unused offer
SK_ID_CURR                                                           
100001                              1.0      NaN     NaN          NaN
100002                              1.0      NaN     NaN          NaN
100003                              3.0      NaN     NaN          NaN

In [14]:
cnt_approved = df['NAME_CONTRACT_TYPE']['Approved']
cnt_canceled = df['NAME_CONTRACT_TYPE']['Canceled']
cnt_refused = df['NAME_CONTRACT_TYPE']['Refused']
cnt_unused = df['NAME_CONTRACT_TYPE']['Unused offer']

** 每个curr_id下approved 的最大最小AMT_CREDT与REFUSED最大最小AMT_CREDT **

In [15]:
df = prev_appli_data[["SK_ID_CURR",\
                      "NAME_CONTRACT_STATUS",\
                      "AMT_CREDIT"]\
                    ].groupby(["SK_ID_CURR",\
                      "NAME_CONTRACT_STATUS"]).max()
df = df.unstack(1)

In [16]:
amt_credit_max_approved = df['AMT_CREDIT']['Approved']
amt_credit_max_refused = df['AMT_CREDIT']['Refused']

In [17]:
df = prev_appli_data[["SK_ID_CURR",\
                      "NAME_CONTRACT_STATUS",\
                      "AMT_CREDIT"]\
                    ].groupby(["SK_ID_CURR",\
                      "NAME_CONTRACT_STATUS"]).min()
df = df.unstack(1)
amt_credit_min_approved = df['AMT_CREDIT']['Approved']
amt_credit_min_refused = df['AMT_CREDIT']['Refused']

** 找到最后Approved的时间与Refused的时间 **

In [18]:
df = prev_appli_data[["SK_ID_CURR",\
                      "NAME_CONTRACT_STATUS",\
                      "DAYS_DECISION"]\
                    ].groupby(["SK_ID_CURR",\
                      "NAME_CONTRACT_STATUS"]).max()
df = df.unstack(1)
days_last_approved = df['DAYS_DECISION']['Approved']
days_last_refused = df['DAYS_DECISION']['Refused']

** 最后的DUE与最后的Ternimation时间**

In [19]:
df = prev_appli_data[["SK_ID_CURR","DAYS_LAST_DUE"]].groupby("SK_ID_CURR").max()
days_last_due = df['DAYS_LAST_DUE']
df = prev_appli_data[["SK_ID_CURR","DAYS_TERMINATION"]].groupby("SK_ID_CURR").max()
days_last_termination = df['DAYS_TERMINATION']


** 求出总的installments_payments中的借还款情况 **

In [20]:
df = instal_pay_data[['SK_ID_CURR','AMT_INSTALMENT','AMT_PAYMENT']].groupby("SK_ID_CURR").sum()


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 339587 entries, 100001 to 456255
Data columns (total 2 columns):
AMT_INSTALMENT    339587 non-null float64
AMT_PAYMENT       339587 non-null float64
dtypes: float64(2)
memory usage: 7.8 MB


In [22]:
clean_data = pd.DataFrame({"CNT_APPROVED":cnt_approved,
                           "CNT_CANCELED":cnt_canceled,
                           "CNT_REFUSED":cnt_refused,
                           "CNT_UNUSED":cnt_unused,
                           "AMT_MAX_APPROVED":amt_credit_max_approved,
                           "AMT_MIN_APPROVED":amt_credit_min_approved,
                           "AMT_MAX_REFUSED":amt_credit_max_refused,
                           "AMT_MIN_REFUSED":amt_credit_min_refused,
                           "DAYS_LAST_APPROVED":days_last_approved,
                           "DAYS_LAST_REFUSED":days_last_refused,
                           "DAYS_LAST_DUE":days_last_due,
                           "DAYS_TERMINATION":days_last_termination})

In [23]:
result = pd.concat([clean_data,df], axis = 1)

In [24]:
result.to_csv(data_dir + '/prev_check1_clean.csv')

In [25]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 340649 entries, 100001 to 456255
Data columns (total 14 columns):
AMT_MAX_APPROVED      337698 non-null float64
AMT_MAX_REFUSED       118277 non-null float64
AMT_MIN_APPROVED      337698 non-null float64
AMT_MIN_REFUSED       118277 non-null float64
CNT_APPROVED          337698 non-null float64
CNT_CANCELED          132694 non-null float64
CNT_REFUSED           118277 non-null float64
CNT_UNUSED            22079 non-null float64
DAYS_LAST_APPROVED    337698 non-null float64
DAYS_LAST_DUE         337340 non-null float64
DAYS_LAST_REFUSED     118277 non-null float64
DAYS_TERMINATION      337340 non-null float64
AMT_INSTALMENT        339587 non-null float64
AMT_PAYMENT           339587 non-null float64
dtypes: float64(14)
memory usage: 39.0 MB


In [26]:
prev_appli_data.loc[prev_appli_data['SK_ID_CURR']==456018]

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
109476,2249277,456018,Revolving loans,4500.000,0.0,90000.0,NaN,NaN,TUESDAY,11,Y,1,NaN,NaN,NaN,XAP,Approved,-1204,XNA,XAP,NaN,Refreshed,XNA,Cards,x-sell,Country-wide,144,Consumer electronics,0.0,XNA,Card X-Sell,-1201.0,-1165.0,365243.0,365243.0,365243.0,0.0
153701,1622362,456018,Cash loans,21274.650,202500.0,202500.0,NaN,202500.0,WEDNESDAY,18,Y,1,NaN,NaN,NaN,XNA,Refused,-1196,Cash through the bank,LIMIT,Unaccompanied,Repeater,XNA,Cash,x-sell,AP+ (Cash loan),8,XNA,12.0,middle,Cash X-Sell: middle,NaN,NaN,NaN,NaN,NaN,NaN
211924,2420717,456018,Cash loans,22544.550,450000.0,533160.0,NaN,450000.0,TUESDAY,12,Y,1,NaN,NaN,NaN,XNA,Refused,-952,Cash through the bank,LIMIT,Unaccompanied,Repeater,XNA,Cash,walk-in,Credit and cash offices,-1,XNA,48.0,middle,Cash Street: middle,NaN,NaN,NaN,NaN,NaN,NaN
701956,1190262,456018,Consumer loans,4536.810,29965.5,28750.5,2997.0,29965.5,SUNDAY,13,Y,1,0.102811,NaN,NaN,XAP,Approved,-2249,Cash through the bank,XAP,NaN,New,Mobile,POS,XNA,Country-wide,31,Connectivity,8.0,high,POS mobile with interest,365243.0,-2186.0,-1976.0,-1976.0,-1969.0,0.0
708586,2767422,456018,Cash loans,44694.810,967500.0,1051789.5,NaN,967500.0,TUESDAY,17,Y,1,NaN,NaN,NaN,Repairs,Refused,-567,Cash through the bank,HC,Unaccompanied,Repeater,XNA,Cash,walk-in,Credit and cash offices,-1,XNA,36.0,low_normal,Cash Street: low,NaN,NaN,NaN,NaN,NaN,NaN
928035,2577090,456018,Cash loans,41585.625,900000.0,978408.0,NaN,900000.0,TUESDAY,19,Y,1,NaN,NaN,NaN,Repairs,Refused,-574,Cash through the bank,HC,Unaccompanied,Repeater,XNA,Cash,walk-in,Credit and cash offices,-1,XNA,36.0,low_normal,Cash Street: low,NaN,NaN,NaN,NaN,NaN,NaN
1333267,1914488,456018,Consumer loans,8505.045,97155.0,112545.0,0.0,97155.0,THURSDAY,17,Y,1,0.000000,NaN,NaN,XAP,Approved,-565,Cash through the bank,XAP,NaN,Repeater,Computers,POS,XNA,Country-wide,30,Consumer electronics,18.0,middle,POS household with interest,365243.0,-534.0,-24.0,-54.0,-51.0,0.0
1412166,1065971,456018,Cash loans,NaN,0.0,0.0,NaN,NaN,TUESDAY,14,Y,1,NaN,NaN,NaN,XNA,Refused,-210,XNA,HC,NaN,Repeater,XNA,XNA,XNA,Credit and cash offices,-1,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN
1423973,1186532,456018,Cash loans,16830.855,252000.0,292914.0,NaN,252000.0,TUESDAY,12,Y,1,NaN,NaN,NaN,XNA,Approved,-952,Cash through the bank,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,36.0,high,Cash X-Sell: high,365243.0,-922.0,128.0,365243.0,365243.0,1.0


In [28]:
#instal_pay_data.loc[instal_pay_data['']]